# Importing the required libraries :

In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Importing the dataset :

In [11]:
df = pd.read_csv('/Users/sudebkumarmandal/Desktop/Movie Recommendations/Movies Recommendation.csv')

In [12]:
df.head(10)

,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,...,Movie_Homepage,Movie_Keywords,Movie_Overview,Movie_Production_House,Movie_Production_Country,Movie_Spoken_Language,Movie_Tagline,Movie_Cast,Movie_Crew,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,09-12-1995,4300000,98.0,6.5,...,NaN,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,"[{'name': 'Allison Anders', 'gender': 1, 'depa...",Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,25-05-1977,775398007,121.0,8.1,...,http://www.starwars.com/films/star-wars-episod...,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,"[{'name': 'George Lucas', 'gender': 2, 'depart...",George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,30-05-2003,940335536,100.0,7.6,...,http://movies.disney.com/finding-nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,06-07-1994,677945399,142.0,8.2,...,NaN,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,"[{'name': 'Alan Silvestri', 'gender': 2, 'depa...",Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,15-09-1999,356296601,122.0,7.9,...,http://www.dreamworks.com/ab/,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...","[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
5,6,Dancer in the Dark,Drama Crime Music,en,12800000,22.022228,17-05-2000,40031879,140.0,7.6,...,NaN,individual dancing usa robbery factory worker,"Selma, a Czech immigrant on the verge of blind...","[{""name"": ""Fine Line Features"", ""id"": 8}, {""na...","[{""iso_3166_1"": ""AR"", ""name"": ""Argentina""}, {""...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",You don't need eyes to see.,Bj\u00f6rk Catherine Deneuve David Morse Peter...,"[{'name': 'Lars von Trier', 'gender': 2, 'depa...",Lars von Trier
6,7,The Fifth Element,Adventure Fantasy Action Thriller Science Fiction,en,90000000,109.528572,07-05-1997,263920180,126.0,7.3,...,NaN,clone taxi cyborg egypt future,"In 2257, a taxi driver is unintentionally give...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""FR"", ""name"": ""France""}]","[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",There is no future without it.,Bruce Willis Gary Oldman Ian Holm Milla Jovovi...,"[{'nam

In [13]:
# COLUMNS :

df.columns

Index(['Movie_ID', 'Movie_Title', 'Movie_Genre', 'Movie_Language',
       'Movie_Budget', 'Movie_Popularity', 'Movie_Release_Date',
       'Movie_Revenue', 'Movie_Runtime', 'Movie_Vote', 'Movie_Vote_Count',
       'Movie_Homepage', 'Movie_Keywords', 'Movie_Overview',
       'Movie_Production_House', 'Movie_Production_Country',
       'Movie_Spoken_Language', 'Movie_Tagline', 'Movie_Cast', 'Movie_Crew',
       'Movie_Director'],
      dtype='object')

In [14]:
# SIZE OF THE DATASET

df.shape

(4760, 21)

In [15]:
# DATA TYPES OF THE COLUMNS OF THE DATASET

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4760 entries, 0 to 4759
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Movie_ID                  4760 non-null   int64  
 1   Movie_Title               4760 non-null   object 
 2   Movie_Genre               4760 non-null   object 
 3   Movie_Language            4760 non-null   object 
 4   Movie_Budget              4760 non-null   int64  
 5   Movie_Popularity          4760 non-null   float64
 6   Movie_Release_Date        4760 non-null   object 
 7   Movie_Revenue             4760 non-null   int64  
 8   Movie_Runtime             4758 non-null   float64
 9   Movie_Vote                4760 non-null   float64
 10  Movie_Vote_Count          4760 non-null   int64  
 11  Movie_Homepage            1699 non-null   object 
 12  Movie_Keywords            4373 non-null   object 
 13  Movie_Overview            4757 non-null   object 
 14  Movie_Pr

In [16]:
df.describe

<bound method NDFrame.describe of       Movie_ID                  Movie_Title                       Movie_Genre  \
0            1                   Four Rooms                      Crime Comedy   
1            2                    Star Wars  Adventure Action Science Fiction   
2            3                 Finding Nemo                  Animation Family   
3            4                 Forrest Gump              Comedy Drama Romance   
4            5              American Beauty                             Drama   
...        ...                          ...                               ...   
4755      4784             Midnight Cabaret                            Horror   
4756      4785             Growing Up Smith               Comedy Family Drama   
4757      4786                       8 Days                    Thriller Drama   
4758      4787              Running Forever                            Family   
4759      4788  To Be Frank, Sinatra at 100                       Documenta

In [17]:
# Fill NaN values with empty strings
df = df.fillna('')

In [18]:
# Combine features into a single string
def combine_features(row):
    return f"{row['Movie_Genre']} {row['Movie_Keywords']} {row['Movie_Overview']} {row['Movie_Cast']} {row['Movie_Director']}"

df['combined_features'] = df.apply(combine_features, axis=1)

In [19]:
# Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [20]:
# Fit and transform the data
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

In [21]:
# Calculate the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [26]:
# Function to get movie recommendations based on a given movie title
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = df[df['Movie_Title'] == title].index[0]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['Movie_Title'].iloc[movie_indices]

In [27]:
# Example usage
print(get_recommendations('Star Wars'))

599                          The Empire Strikes Back
600                               Return of the Jedi
603     Star Wars: Episode III - Revenge of the Sith
1113                                   Shanghai Noon
1418                                 The Last Legion
601        Star Wars: Episode I - The Phantom Menace
3323                             The Helix... Loaded
1592                                 The Ice Pirates
2563                    The Fall of the Roman Empire
419                               Lawrence of Arabia
Name: Movie_Title, dtype: object
